In [1]:
import pandas as pd
import os
import glob
from datetime import datetime, timedelta
import matplotlib as plt
from shapely.geometry import Point
from shapely.geometry import Polygon
import ast


In [3]:
folder_path = '../data/raw/'

# Get a list of all the CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

## First Preprocessing

time_new and day_new for better readability
<br> no_movement column shows episodes without movement for at least 2min

In [8]:
def Preprocess(df):
    df.columns = df.columns.str.lower()

    df["hour"] = df["time"].astype(int)
    df["minute"] = (60 * (df["time"] - df["hour"])).astype(int)
    df['minute'] = df['minute'].apply(lambda x: f"0{x}" if x < 10 else str(x))  # fill minutes with 0
    df['time_new'] = df['hour'].astype('str') + ':' + df['minute'].astype('str')

    df["day_new"] = df["day"] % 5

    day_old = 0
    week_number = 0

    for i in range(len(df)):
        df.loc[i, "day_new"] = df["day"][i] % 5

        day_new = df["day"][i] % 5
        if day_new < day_old:
            week_number += 1
        day_old = df["day"][i] % 5
        df.loc[df.index == i, 'week_num'] = week_number

        if(i < len(df)-1): 
            duration = (df["time"][i+1] - df["time"][i]) * 60
            if (duration >= 5):
                df.loc[i, "no_movement_5min"] = 1
            else:
                df.loc[i, "no_movement_5min"] = 0
        if(i < len(df)-1): 
            duration = (df["time"][i+1] - df["time"][i]) * 60
            if (duration >= 15):
                df.loc[i, "no_movement_15min"] = 1
            else:
                df.loc[i, "no_movement_15min"] = 0


    df['time_new'] = pd.to_datetime(df['time_new'], format='%H:%M').dt.time

    start_date = datetime(2023, 1, 1)
    df['datetime'] = df.apply(lambda row: datetime.combine(start_date + timedelta(days=row['day']), row['time_new']),
                            axis=1)

    start_time = min(df["datetime"])
    end_time = max(df["datetime"])
    datetime_column = pd.date_range(start=start_time, end=end_time, freq="1T")
    datetime_df = pd.DataFrame({"datetime": datetime_column})

    datetime_df['date'] = datetime_df['datetime'].dt.date
    unique_dates = datetime_df['date'].unique()
    unique_days = df["day"].unique()
    l = []
    for day, dates in enumerate(unique_dates):
        date_df = datetime_df[datetime_df['datetime'].dt.date == dates]
        try:
            day_df = df.loc[df["day"] == day]
            start_time = min(day_df["time_new"])
            end_time = max(day_df["time_new"])
        except ValueError:
            continue
        l.append(datetime_df[(datetime_df['datetime'].dt.time >= start_time)
            & (datetime_df['datetime'].dt.time <= end_time)])

    df_date = l[0]
    new_df = df.merge(df_date, on="datetime", how="right")
    

    new_df["y_binary"] = ""
    new_df.loc[(new_df["y"] >= -1) & (new_df["y"] <= 3), "y_binary"] = 1
    new_df.loc[(new_df["y"] >= -12) & (new_df["y"] < -1), "y_binary"] = 0
    #new_df = new_df.loc[new_df["y_binary"].isin([0, 1])]
    new_df = new_df.fillna(method="ffill")
    res = new_df[["day", "time", "time_new", "day_new", "week_num", "datetime", "x", "y", "z", "y_binary", "no_movement_5min", "no_movement_15min"]]

    return res

In [30]:
# Get room to tupel (x,y,z)
def getRoom(df):
    room_coordinates = pd.read_csv('../data/preprocessed/Raumkoordinaten/raumkoordinaten_prep_unity.csv', index_col = 0)
    df.columns = df.columns.str.lower()
    df["room"] = ""

    if 'y_binary' in df:
        pass
    else:
        df["y_binary"] = ""
        df.loc[(df["y"] >= -1) & (df["y"] <= 0), "y_binary"] = 1
        df.loc[(df["y"] >= -10) & (df["y"] <= -6), "y_binary"] = 0
        df = df.loc[df["y_binary"].isin([0,1])]
        df = df.reset_index(drop=True)
    
    

    for i in range(len(df["x"])):
        x = df["x"][i]
        z = df["z"][i]
        floor = df["y_binary"][i]
        coor_help = room_coordinates.loc[room_coordinates["floor"] == floor]
        coor_help = coor_help.reset_index(drop=True)

        point = Point(x, z)
        for c in range(len(coor_help)):
            polygon = Polygon(ast.literal_eval(coor_help["coor"][c]))
            res = polygon.contains(point)
            if res:
                df.loc[i, "room"] = coor_help["room"][c]
    return df
    

    

In [6]:
folder_path = '../data/raw/'

# Get a list of all the CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

In [ ]:
for i in csv_files:
    filename = os.path.splitext(os.path.basename(i))[0]
    a = pd.read_csv("../data/raw/" + filename + ".csv")
    data_new = Preprocess(a)
    a.to_csv('../data/preprocessed/v2/' + filename + "_prep_v2.csv")    
    print(filename)
    